# Quick-Start
Following documentation available here
https://stmorse.github.io/journal/spotify-api.html 



Source code for project available on github here 
https://github.com/spotify/web-api-auth-examples


In [1]:
import requests
import pandas as pd


# Taken from Spotify App Dashboard, secret may auto-rotate.
CLIENT_ID = "324585f433a84680a7a7a6414c721d13"
CLIENT_SECRET = ""

In [2]:
AUTH_URL = "https://accounts.spotify.com/api/token"

# Post request for client credentials
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET
    })

# Convert response to json
auth_response_data = auth_response.json()

# Save the access token
access_token = auth_response_data['access_token']

print(auth_response_data)
# Note: Access token expires in 3600sec? = 1 hour?

{'access_token': 'BQBDUCB37kUVKtJhRNI74T5QIEetFiZWVvTA6DujlxjmCQIYewcsKbnNwmEIRYyv4OZeRvTAQqhy2tecEcE', 'token_type': 'Bearer', 'expires_in': 3600}


In [3]:
headers = {
    'Authorization' : 'Bearer {token}'.format(token=access_token)
}

# Testing here with single track

In [4]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID for Classic-MKTO
# https://open.spotify.com/track/6FE2iI43OZnszFLuLtvvmg?si=e4e75d9c000a4219
track_id = '6FE2iI43OZnszFLuLtvvmg'

# Make GET request using header for access token
r = requests.get(BASE_URL + 'audio-features/'+track_id, headers=headers)
r=r.json()

In [5]:
display(r)
norm_params = ['danceability', 'energy', 'speechiness', 'acousticness',\
               'instrumentalness', 'liveness', 'valence']

extra_params = ['tempo', 'mode', 'duration']

for i in norm_params:
    print("%s = %.4f" %(i, r[i]))

{'danceability': 0.72,
 'energy': 0.791,
 'key': 1,
 'loudness': -4.689,
 'mode': 1,
 'speechiness': 0.124,
 'acousticness': 0.0384,
 'instrumentalness': 0,
 'liveness': 0.157,
 'valence': 0.756,
 'tempo': 102.071,
 'type': 'audio_features',
 'id': '6FE2iI43OZnszFLuLtvvmg',
 'uri': 'spotify:track:6FE2iI43OZnszFLuLtvvmg',
 'track_href': 'https://api.spotify.com/v1/tracks/6FE2iI43OZnszFLuLtvvmg',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6FE2iI43OZnszFLuLtvvmg',
 'duration_ms': 175427,
 'time_signature': 4}

danceability = 0.7200
energy = 0.7910
speechiness = 0.1240
acousticness = 0.0384
instrumentalness = 0.0000
liveness = 0.1570
valence = 0.7560


In [22]:
# Red Hot Chili Peppers 
#https://open.spotify.com/artist/0L8ExT028jH3ddEcZwqJJ5?si=c2b222b0f949459a
artist_id = '0L8ExT028jH3ddEcZwqJJ5'

# Pull all artists albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums',
                headers=headers,
                params = {'include_groups': 'album', 'limit': 50})

d = r.json()

In [43]:
display(d['items'][0]['name'])
display(d['items'][0]['id'])
#display(d['items'][0])

for album in d['items']:
    print(album['name'], "---", album['release_date'])

'The Getaway'

'43otFXrY0bgaq5fB3GrZj6'

The Getaway --- 2016-06-17
I'm with You --- 2011-08-29
Stadium Arcadium --- 2006-05-09
By the Way --- 2002-07-09
By the Way (Deluxe Edition) --- 2002-07-09
Californication --- 1999-06-08
Californication (Deluxe Edition) --- 1999-06-08
One Hot Minute (Deluxe Edition) --- 1995-09-12
One Hot Minute --- 1995-09-12
Blood Sugar Sex Magik --- 1991-09-24
Blood Sugar Sex Magik (Deluxe Edition) --- 1991-09-24
Mother's Milk --- 1989-08-16
The Uplift Mofo Party Plan --- 1987
Freaky Styley --- 1985-08-16
Red Hot Chili Peppers --- 1984-08-10


In [46]:
data = [] # will hold all track info
albums = [] # to keep track of duplicate albums

for album in d['items']:
    album_name = album['name']
    
    #Shorten track name to first bracket... Kinda works
    trim_name = album_name.split('(')[0].strip()
    if trim_name.upper() in albums:
        continue
    albums.append(trim_name.upper()) #upper() for all caps
    
    # keep track of progress
    print(album_name)
    
    
    #pull all tracks from this album
    r = requests.get(BASE_URL + 'albums/'+album['id'] + '/tracks',
                    headers = headers)
    
    tracks = r.json()['items']
    
    for track in tracks:
        # get audio features
        f = requests.get(BASE_URL + 'audio-features/'+track['id'],
                        headers=headers)
        
        f=f.json()
        
        # combine features json with album info
        f.update({
            'track_name': track['name'],
            'album_name': album_name,
            'short_album_name': trim_name,
            'release_date': album['release_date'],
            'album_id': album['id']
            
        })
        
        data.append(f)
                                 
                                 
display(f)
df = pd.DataFrame(data)

SyntaxError: invalid syntax (<ipython-input-46-7dc32ab5fad5>, line 9)

In [47]:
# Last track
display(f)

{'danceability': 0.658,
 'energy': 0.598,
 'key': 2,
 'loudness': -12.971,
 'mode': 1,
 'speechiness': 0.184,
 'acousticness': 0.747,
 'instrumentalness': 0.144,
 'liveness': 0.143,
 'valence': 0.875,
 'tempo': 109.663,
 'type': 'audio_features',
 'id': '2yqPd7tLYAN0V4rzUIRUxh',
 'uri': 'spotify:track:2yqPd7tLYAN0V4rzUIRUxh',
 'track_href': 'https://api.spotify.com/v1/tracks/2yqPd7tLYAN0V4rzUIRUxh',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2yqPd7tLYAN0V4rzUIRUxh',
 'duration_ms': 237867,
 'time_signature': 4,
 'track_name': "What It Is (AKA Nina's Song) - Demo Version",
 'album_name': 'Red Hot Chili Peppers',
 'short_album_name': 'Red Hot Chili Peppers',
 'release_date': '1984-08-10',
 'album_id': '5QxFTj8ar3blXVyVEo2rxD'}

In [65]:
df = pd.DataFrame(data)
df.head(8)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,short_album_name,release_date,album_id
0,0.797,0.642,0,-8.082,1,0.0725,0.0799,0.000012,0.371,0.667,...,spotify:track:3bIQIx7hveYPQDdhjZ1kcq,https://api.spotify.com/v1/tracks/3bIQIx7hveYP...,https://api.spotify.com/v1/audio-analysis/3bIQ...,250387,4,The Getaway,The Getaway,The Getaway,2016-06-17,43otFXrY0bgaq5fB3GrZj6
1,0.700,0.742,5,-6.777,0,0.0716,0.0722,0.019900,0.110,0.197,...,spotify:track:2oaK4JLVnmRGIO9ytBE1bt,https://api.spotify.com/v1/tracks/2oaK4JLVnmRG...,https://api.spotify.com/v1/audio-analysis/2oaK...,302000,4,Dark Necessities,The Getaway,The Getaway,2016-06-17,43otFXrY0bgaq5fB3GrZj6
2,0.610,0.872,2,-5.428,0,0.0422,0.1160,0.147000,0.152,0.842,...,spotify:track:0cv2LgkvEoQiGgFWcZaAMA,https://api.spotify.com/v1/tracks/0cv2LgkvEoQi...,https://api.spotify.com/v1/audio-analysis/0cv2...,200467,4,We Turn Red,The Getaway,The Getaway,2016-06-17,43otFXrY0bgaq5fB3GrZj6
3,0.476,0.511,8,-6.250,1,0.0324,0.3190,0.000129,0.119,0.339,...,spotify:track:0pjCkLjbgSLn5c0Ilwuv8z,https://api.spotify.com/v1/tracks/0pjCkLjbgSLn...,https://api.spotify.com/v1/audio-analysis/0pjC...,211520,4,The Longest Wave,The Getaway,The Getaway,2016-06-17,43otFXrY0bgaq5fB3GrZj6
4,0.365,0.804,9,-5.922,0,0.1050,0.1040,0.000066,0.150,0.577,...,spotify:track:2XTkpF9T2PKvcLgamGJGx1,https://api.spotify.com/v1/tracks/2XTkpF9T2PKv...,https://api.spotify.com/v1/audio-analysis/2XTk...,268733,4,Goodbye Angels,The Getaway,The Getaway,2016-06-17,43otFXrY0bgaq5fB3GrZj6
5,0.591,0.794,4,-5.935,0,0.0690,0.1830,0.000000,0.292,0.875,...,spotify:track:6GsP3uMCd0Dn5T37C93waZ,https://api.spotify.com/v1/tracks/6GsP3uMCd0Dn...,https://api.spotify.com/v1/audio-analysis/6GsP...,221440,4,Sick Love,The Getaway,The Getaway,2016-06-17,43otFXrY0bgaq5fB3GrZj6
6,0.852,0.651,8,-6.254,1,0.0411,0.0432,0.019200,0.158,0.960,...,spotify:track:5m578MgxuEC956dulprCe8,https://api.spotify.com/v1/tracks/5m578MgxuEC9...,https://api.spotify.com/v1/audio-analysis/5m57...,263800,4,Go Robot,The Getaway,The Getaway,2016-06-17,43otFXrY0bgaq5fB3GrZj6
7,0.670,0.750,6,-5.502,0,0.0583,0.1370,0.000004,0.197,0.880,...,spotify:track:0SMzXgUFqcAt0WNCsGm3tM,https://api.spotify.com/v1/tracks/0SMzXgUFqcAt...,https://api.spotify.com/v1/audio-analysis/0SMz...,202933,4,Feasting on the Flowers,The Getaway,The Getaway,2016-06-17,43otFXrY0bgaq5fB3GrZj6


In [63]:
norm_params = ['danceability', 'energy', 'speechiness', 'acousticness',\
               'instrumentalness', 'liveness', 'valence']

extra_params = ['tempo', 'mode', 'duration']

for i in norm_params:
    print("Mean %s = %f" %(i, df[i].mean()) )

Mean danceability = 0.546176
Mean energy = 0.826364
Mean speechiness = 0.080515
Mean acousticness = 0.072179
Mean instrumentalness = 0.074861
Mean liveness = 0.188045
Mean valence = 0.501890
